In [1]:
# Ensure python 3 compatibility
from __future__ import division, print_function, absolute_import

# Import necessary libraries:
# General utilities:
import sys
import os
import math

# Computation:
import numpy as np
import h5py

# Visualization:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import ipywidgets as widgets

from IPython.display import display

# Finally, pycroscopy itself
import pycroscopy as px

from os.path import join as pjoin

# set up notebook to show plots within the notebook
% matplotlib inline

import glob

#import moviepy.video.io.ImageSequenceClip
#try:
#    output = subprocess.check_output(['ffmpeg', '-version'])
#    version = output.split(b'\n')[0].split()[2]
#    print('Found: ffmpeg v{}'.format(version.decode('utf-8')))
#    ffmpeg_installed = True
#except:
 #   ffmpeg_installed = False

In [2]:
%%file custom.mplstyle

axes.linewidth: 1.5
xtick.major.size: 6
xtick.minor.size: 2
xtick.major.width: 1.5
xtick.minor.width: 1.5
ytick.major.size: 6
ytick.minor.size: 2
ytick.major.width: 1.5
ytick.minor.width: 1.5
axes.labelweight: bold
axes.labelpad: 1
axes.labelsize: 12
xtick.major.pad: 1
ytick.major.pad: 1
xtick.labelsize: 12
ytick.labelsize: 12
xtick.top: True
ytick.right: True
xtick.direction: in
ytick.direction: in
image.interpolation: nearest
    
# Loads the custom style
plt.style.use('./custom.mplstyle')

Overwriting custom.mplstyle


In [2]:
plt.style.use('./custom.mplstyle')


C:\Users\Josh_\Anaconda3\lib\site-packages\matplotlib\__init__.py:955: UserWarning: Illegal line #25
	"plt.style.use('./custom.mplstyle')"
	in file "./custom.mplstyle"
  warnings.warn('Illegal %s' % error_details)


In [3]:
 input_file_path ='D:/PZT001_BEPs_0003.h5' #px.io_utils.uiGetFile(caption='Select translated .h5 file or raw experiment data',
                                         #filter='Translated file (*.h5);; \\ Parameters for raw BE data (*.txt *.mat *xls *.xlsx)')

(data_dir, data_name) = os.path.split(input_file_path)

if input_file_path.endswith('.h5'):
    # No translation here
    h5_path = input_file_path
    tl = px.LabViewH5Patcher()
    hdf = tl.translate(h5_path)
else:
    # Set the data to be translated
    data_path = input_file_path

    (junk, base_name) = os.path.split(data_dir)

    # Check if the data is in the new or old format.  Initialize the correct translator for the format.
    if base_name == 'newdataformat':
        (junk, base_name) = os.path.split(junk)
        translator = px.BEPSndfTranslator(max_mem_mb=max_mem)
    else:
        translator = px.BEodfTranslator(max_mem_mb=max_mem)
    if base_name.endswith('_d'):
        base_name = base_name[:-2]
    # Translate the data
    h5_path = translator.translate(data_path, show_plots=True, save_plots=False)
    tl = px.LabViewH5Patcher()
    hdf = tl.translate(h5_path)
print('Working on:\n' + h5_path)

h5_main = px.hdf_utils.getDataSet(hdf.file, 'Raw_Data')[0]

File is already Pycroscopy ready.
Working on:
D:/PZT001_BEPs_0003.h5


In [4]:
h5_sho_group = px.hdf_utils.findH5group(h5_main, 'SHO_Fit')
sho_fitter = px.BESHOmodel(h5_main, parallel=True)
if len(h5_sho_group) == 0:
    print('No SHO fit found. Doing SHO Fitting now')
    h5_sho_guess = sho_fitter.do_guess(strategy='complex_gaussian', processors=max_cores)
    h5_sho_fit = sho_fitter.do_fit(processors=max_cores)
else:
    print('Taking previous SHO results already present in file')
    h5_sho_guess = h5_sho_group[-1]['Guess']
    try:
        h5_sho_fit = h5_sho_group[-1]['Fit']
    except KeyError:
        print('Previously computed guess found. Now computing fit')
        h5_sho_fit = sho_fitter.do_fit(processors=max_cores, h5_guess=h5_sho_guess)

Taking previous SHO results already present in file


In [ ]:
# Specify output file path
output_file_path = './'

# If HV amplifier was used set high_voltage_amplf to 10, else to 1
high_voltage_amplf = 10


In [5]:
(Nd_mat, success) = px.io.hdf_utils.reshape_to_Ndims(h5_sho_fit)
print('Reshape Success = ' + str(success))
print('Nd_mat shape = ', Nd_mat.shape)

phase_offset = Nd_mat[0, 0, 1, 0, 0]['Phase [rad]']

# phase_offset = 0;

print('Phase offset [rad] = ', phase_offset)

Nd_mat[:,:,:,:,:]['Phase [rad]'] = Nd_mat[:,:,:,:,:]['Phase [rad]'] - phase_offset

Reshape Success = True
Nd_mat shape =  (80, 80, 128, 2, 3)
Phase offset [rad] =  0.342565


In [ ]:
#Nd_mat["Phase [rad]"].shape[2]

In [6]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from scipy import io
import numpy as np
import pycroscopy as px
import matplotlib.pyplot as plt
from os.path import join as pjoin
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential, Input, Model
from keras.layers import (Dense, Conv1D, Convolution2D, GRU, LSTM, Recurrent, Bidirectional, TimeDistributed,
                          Dropout, Flatten, RepeatVector, Reshape, MaxPooling1D, UpSampling1D, BatchNormalization)
from keras.layers.core import Lambda
from keras.optimizers import Adam
import os
from os.path import join as pjoin
from keras.regularizers import l1
import sys


% matplotlib inline

Using TensorFlow backend.


In [ ]:
#len(qf_on.reshape(-1))

In [7]:
def norm(indat):
    indat-=np.mean(indat)
    indat/=np.std(indat)
    return indat

In [8]:
amp_off = np.atleast_3d(Nd_mat["Amplitude [V]"][:,:,:,1,2].reshape(-1,128))
phase_off = np.atleast_3d(Nd_mat["Phase [rad]"][:,:,:,1,2].reshape(-1,128))
freq_off = np.atleast_3d(Nd_mat["Frequency [Hz]"][:,:,:,1,2].reshape(-1,128))
qf_off = np.atleast_3d(Nd_mat["Quality Factor"][:,:,:,1,2].reshape(-1,128))

amp_on = np.atleast_3d(Nd_mat["Amplitude [V]"][:,:,:,0,2].reshape(-1,128))
phase_on = np.atleast_3d(Nd_mat["Phase [rad]"][:,:,:,0,2].reshape(-1,128))
freq_on = np.atleast_3d(Nd_mat["Frequency [Hz]"][:,:,:,0,2].reshape(-1,128))
qf_on= np.atleast_3d(Nd_mat["Quality Factor"][:,:,:,0,2].reshape(-1,128))

In [9]:
amp_off = np.atleast_3d(norm(amp_off))
phase_off = np.atleast_3d(norm(phase_off))
freq_off = np.atleast_3d(norm(freq_off))
qf_off = np.atleast_3d(norm(qf_off))

amp_on = np.atleast_3d(norm(amp_on))
phase_on = np.atleast_3d(norm(phase_on))
freq_on = np.atleast_3d(norm(freq_on))
qf_on= np.atleast_3d(norm(qf_on))

In [10]:
for i in range(len(qf_on.reshape(-1))):
    if qf_on.reshape(-1)[i] > 900:
        qf_on.reshape(-1)[i] = np.mean(qf_on)
        
for i in range(len(qf_off.reshape(-1))):
    if qf_off.reshape(-1)[i] > 900:
        qf_off.reshape(-1)[i] = np.mean(qf_off)

In [ ]:
#np.std((Nd_mat["Amplitude [V]"][:,:,:,1,2].reshape(-1)))

In [ ]:
fig = plt.figure(figsize=(24, 18))
ax_amp_off = plt.subplot2grid((2, 4), (0, 0), colspan=1, rowspan=1)
ax_amp_on = plt.subplot2grid((2, 4), (0, 1), colspan=1, rowspan=1)
ax_phase_off = plt.subplot2grid((2, 4), (0, 2), colspan=1, rowspan=1)
ax_phase_on = plt.subplot2grid((2, 4), (0, 3), colspan=1, rowspan=1)
ax_freq_off = plt.subplot2grid((2, 4), (1, 0), colspan=1, rowspan=1)
ax_freq_on = plt.subplot2grid((2, 4), (1, 1), colspan=1, rowspan=1)
ax_qf_off = plt.subplot2grid((2, 4), (1, 2), colspan=1, rowspan=1)
ax_qf_on = plt.subplot2grid((2, 4), (1, 3), colspan=1, rowspan=1)


ax_amp_off.hist(amp_off.reshape(-1),100)
ax_amp_on.hist(amp_on.reshape(-1),100)
ax_phase_off.hist(phase_off.reshape(-1),100)
ax_phase_on.hist(phase_on.reshape(-1),100)
ax_freq_off.hist(freq_off.reshape(-1),100) 
ax_freq_on.hist(freq_on.reshape(-1),100)
ax_qf_off.hist(qf_off.reshape(-1),100)
ax_qf_on.hist(qf_on.reshape(-1),100)
    
    
    
    
#(amp_on.reshape(-1),100)


In [11]:
## PCA of imput data to remove noise
import sklearn

In [12]:
# set the number of principle components
PC_number = 20
pca = sklearn.decomposition.PCA(n_components=PC_number)
pca_freq = pca.fit(freq_off[:,:,0])
pca_recon_freq = pca.inverse_transform(pca.transform(freq_off[:,:,0]))

In [13]:
pca_input_freq_off = np.atleast_3d(pca_recon_freq[:,::2])

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(18,18))

axes[0].plot(np.linspace(1, PC_number, PC_number),
             np.cumsum(pca_freq.explained_variance_ratio_) * 100,
             '-ok')
t = np.random.randint(0,3600)
axes[1].plot(freq_off[t,:])
axes[1].plot(pca_recon_freq[t,:])


In [14]:
def make_folder(folder_name, root='./'):

    folder = pjoin(root, '{}'.format(folder_name))
    os.makedirs(folder, exist_ok=True)

    return (folder)

In [15]:
def network1(inp):
    out = Bidirectional(LSTM(128, return_sequences=False))(inp)
    en_out_a_off = Dense(16, activation='relu')(out)
    x = RepeatVector(64)(en_out_a_off)   
    #x = TimeDistributed(Dense(1, activation='linear'))(en_out_a_off)
    x = Bidirectional(LSTM(128, activation= "sigmoid", return_sequences=True))(x)
    x = TimeDistributed(Dense(1, activation='linear'))(x)
    return x

In [16]:
def network2(inp):
    out = Bidirectional(LSTM(128, return_sequences=True))(inp)
    en_out_a_off = Dense(16, activation='relu')(out)
   # x = RepeatVector(64)(en_out_a_off)   
    #x = TimeDistributed(Dense(1, activation='linear'))(en_out_a_off)
    x = Bidirectional(LSTM(128, activation= "sigmoid", return_sequences=True))(en_out_a_off)
    x = TimeDistributed(Dense(1, activation='linear'))(x)
    return x

In [17]:
def network3(inp):
    out = Bidirectional(LSTM(128,activation = "sigmoid", return_sequences=True))(inp)
    x = TimeDistributed(Dense(1, activation="linear"))(out)
    return x

In [32]:
mods = keras.models.load_model("D:/Layerwise/freq_only4/weights.4999-0.01.hdf5")
mods_2 =keras.models.load_model("D:/Layerwise/freq_second/weights.5000-0.01.hdf5")
mods_3 = keras.models.load_model("D:/Layerwise/freq_third/weights.5000-0.01.hdf5")
mods_4 = keras.models.load_model("C:/Users/Josh_/CKPFM_Doped_001_PZT/freq_encode/weights.5000-0.01.hdf5")

In [19]:

input_off_amp =Input(shape=(Nd_mat["Phase [rad]"].shape[2]//2,1))
x = network2(input_off_amp)
gen = Model(inputs=[input_off_amp], 
              outputs=x)

gen.compile(optimizer=Adam(3e-5),loss='mse')#

In [20]:
lbs_list = mods.get_weights()
gen.set_weights(lbs_list)

In [21]:
def get_ith_layer_output(model, X, i, mode='test'):
    ''' see https://keras.io/getting-started/faq/#keras-faq-frequently-asked-keras-questions'''
    get_ith_layer = keras.backend.function(
        [model.layers[0].input, keras.backend.learning_phase()], [model.layers[i].output])
    layer_output = get_ith_layer([X, 0 if mode=='test' else 1])[0]
    return layer_output

In [22]:
one = get_ith_layer_output(gen,pca_input_freq_off,1)

In [23]:
input_layer_1=Input(shape=(one.shape[1:]))
x = network2(input_layer_1)
gen_2 = Model(inputs=[input_layer_1], 
              outputs=x)

gen_2.compile(optimizer=Adam(3e-5),loss='mse')#

In [24]:
lbs_2= mods_2.get_weights()
gen_2.set_weights(lbs_2)

In [25]:
two = get_ith_layer_output(gen_2,one,1)

In [26]:
input_layer_1=Input(shape=(two.shape[1:]))
x = network1(input_layer_1)
model = Model(inputs=[input_layer_1], 
              outputs=x)
model.compile(optimizer=Adam(3e-5),loss='mse')#

In [27]:
lbs_3 = mods_3.get_weights()
model.set_weights(lbs_3)

In [28]:
# get the ith layer
three = get_ith_layer_output(model, two, 4)

In [29]:
# create final layer with ith layer input

input_decode=Input(shape=(three.shape[1:]))
x = network3(input_decode)
decode = Model(inputs=[input_decode],
              outputs=x)
decode.compile(optimizer=Adam(3e-5),loss='mse')

In [33]:
lbs_4 = mods_4.get_weights()
decode.set_weights(lbs_4)

In [34]:
four = get_ith_layer_output(decode, three, 1)

In [35]:

input_decode=Input(shape=(three.shape[1:]))
x = network3(input_decode)
decode_2 = Model(inputs=[input_decode],
              outputs=x)
decode_2.compile(optimizer=Adam(3e-5),loss='mse')

In [36]:
Path = 'freq_decode_2'
make_folder(Path)
tbCallBack = keras.callbacks.TensorBoard(
    log_dir='./' + Path, histogram_freq=0, write_graph=True, write_images=True)
filepath = './' + Path + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, verbose=0, save_best_only=False,
                                             save_weights_only=False, mode='min', period=1)
#from keras.utils import multi_gpu_model

# Replicates `model` on 2 GPUs.
# This assumes that your machine has 2 available GPUs.
#parallel_model = multi_gpu_model(model, gpus=2)
#parallel_model.compile(optimizer=Adam(3e-5),loss='mse')

decode_2.fit([four], 
            [pca_input_freq_off],
          validation_data=([four], 
          [pca_input_freq_off]),
          epochs=5000,batch_size=1024, callbacks=[tbCallBack, checkpoint])

Train on 6400 samples, validate on 6400 samples
Epoch 1/5000
6400/6400 [==============================] - 7s 1ms/step - loss: 1.0311 - val_loss: 1.0044
Epoch 2/5000
6400/6400 [==============================] - 2s 387us/step - loss: 1.0005 - val_loss: 0.9961
Epoch 3/5000
6400/6400 [==============================] - 2s 388us/step - loss: 0.9896 - val_loss: 0.9779
Epoch 4/5000
6400/6400 [==============================] - 2s 389us/step - loss: 0.9714 - val_loss: 0.9621
Epoch 5/5000
6400/6400 [==============================] - 2s 388us/step - loss: 0.9568 - val_loss: 0.9465
Epoch 6/5000
6400/6400 [==============================] - 2s 386us/step - loss: 0.9397 - val_loss: 0.9296
Epoch 7/5000
6400/6400 [==============================] - 2s 388us/step - loss: 0.9239 - val_loss: 0.9138
Epoch 8/5000
6400/6400 [==============================] - 2s 388us/step - loss: 0.9070 - val_loss: 0.8967
Epoch 9/5000
6400/6400 [==============================] - 2s 388us/step - loss: 0.8907 - val_loss: 0.8812


6400/6400 [==============================] - 3s 395us/step - loss: 0.0268 - val_loss: 0.0266
Epoch 78/5000
6400/6400 [==============================] - 3s 396us/step - loss: 0.0266 - val_loss: 0.0265
Epoch 79/5000
6400/6400 [==============================] - 3s 409us/step - loss: 0.0264 - val_loss: 0.0263
Epoch 80/5000
6400/6400 [==============================] - 3s 401us/step - loss: 0.0262 - val_loss: 0.0260
Epoch 81/5000
6400/6400 [==============================] - 3s 406us/step - loss: 0.0260 - val_loss: 0.0258
Epoch 82/5000
6400/6400 [==============================] - 3s 416us/step - loss: 0.0258 - val_loss: 0.0256
Epoch 83/5000
6400/6400 [==============================] - 3s 400us/step - loss: 0.0256 - val_loss: 0.0254
Epoch 84/5000
6400/6400 [==============================] - 3s 405us/step - loss: 0.0254 - val_loss: 0.0253
Epoch 85/5000
6400/6400 [==============================] - 3s 400us/step - loss: 0.0252 - val_loss: 0.0251
Epoch 86/5000
6400/6400 [==========================

6400/6400 [==============================] - 3s 421us/step - loss: 0.0174 - val_loss: 0.0174
Epoch 154/5000
6400/6400 [==============================] - 3s 418us/step - loss: 0.0174 - val_loss: 0.0173
Epoch 155/5000
6400/6400 [==============================] - 3s 432us/step - loss: 0.0173 - val_loss: 0.0173
Epoch 156/5000
6400/6400 [==============================] - 3s 441us/step - loss: 0.0172 - val_loss: 0.0172
Epoch 157/5000
6400/6400 [==============================] - 3s 437us/step - loss: 0.0172 - val_loss: 0.0171
Epoch 158/5000
6400/6400 [==============================] - 3s 416us/step - loss: 0.0171 - val_loss: 0.0171
Epoch 159/5000
6400/6400 [==============================] - 3s 415us/step - loss: 0.0170 - val_loss: 0.0170
Epoch 160/5000
6400/6400 [==============================] - 3s 429us/step - loss: 0.0170 - val_loss: 0.0169
Epoch 161/5000
6400/6400 [==============================] - 3s 437us/step - loss: 0.0169 - val_loss: 0.0169
Epoch 162/5000
6400/6400 [=================

Epoch 229/5000
6400/6400 [==============================] - 3s 423us/step - loss: 0.0142 - val_loss: 0.0142
Epoch 230/5000
6400/6400 [==============================] - 3s 432us/step - loss: 0.0142 - val_loss: 0.0142
Epoch 231/5000
6400/6400 [==============================] - 3s 423us/step - loss: 0.0142 - val_loss: 0.0142
Epoch 232/5000
6400/6400 [==============================] - 3s 423us/step - loss: 0.0142 - val_loss: 0.0142
Epoch 233/5000
6400/6400 [==============================] - 3s 424us/step - loss: 0.0142 - val_loss: 0.0141
Epoch 234/5000
6400/6400 [==============================] - 3s 534us/step - loss: 0.0141 - val_loss: 0.0141
Epoch 235/5000
6400/6400 [==============================] - 3s 419us/step - loss: 0.0141 - val_loss: 0.0141
Epoch 236/5000
6400/6400 [==============================] - 3s 425us/step - loss: 0.0141 - val_loss: 0.0141
Epoch 237/5000
6400/6400 [==============================] - 3s 418us/step - loss: 0.0141 - val_loss: 0.0140
Epoch 238/5000
6400/6400 [==

Epoch 305/5000
6400/6400 [==============================] - 3s 431us/step - loss: 0.0129 - val_loss: 0.0129
Epoch 306/5000
6400/6400 [==============================] - 3s 435us/step - loss: 0.0129 - val_loss: 0.0129
Epoch 307/5000
6400/6400 [==============================] - 3s 443us/step - loss: 0.0129 - val_loss: 0.0129
Epoch 308/5000
6400/6400 [==============================] - 3s 418us/step - loss: 0.0129 - val_loss: 0.0129
Epoch 309/5000
6400/6400 [==============================] - 3s 433us/step - loss: 0.0129 - val_loss: 0.0129
Epoch 310/5000
6400/6400 [==============================] - 3s 429us/step - loss: 0.0129 - val_loss: 0.0129
Epoch 311/5000
6400/6400 [==============================] - 3s 431us/step - loss: 0.0129 - val_loss: 0.0128
Epoch 312/5000
6400/6400 [==============================] - 3s 429us/step - loss: 0.0128 - val_loss: 0.0128
Epoch 313/5000
6400/6400 [==============================] - 3s 426us/step - loss: 0.0128 - val_loss: 0.0128
Epoch 314/5000
6400/6400 [==

...

6400/6400 [==============================] - 3s 405us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2040/5000
6400/6400 [==============================] - 3s 408us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2041/5000
6400/6400 [==============================] - 3s 407us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2042/5000
6400/6400 [==============================] - 3s 422us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2043/5000
6400/6400 [==============================] - 3s 502us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2044/5000
6400/6400 [==============================] - 3s 424us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2045/5000
6400/6400 [==============================] - 3s 410us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2046/5000
6400/6400 [==============================] - 3s 408us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2047/5000
6400/6400 [==============================] - 3s 416us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2048/5000
6400/6400 [========

6400/6400 [==============================] - 3s 419us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2115/5000
6400/6400 [==============================] - 3s 420us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2116/5000
6400/6400 [==============================] - 3s 411us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2117/5000
6400/6400 [==============================] - 3s 409us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2118/5000
6400/6400 [==============================] - 3s 402us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2119/5000
6400/6400 [==============================] - 3s 406us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2120/5000
6400/6400 [==============================] - 3s 514us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2121/5000
6400/6400 [==============================] - 3s 410us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2122/5000
6400/6400 [==============================] - 3s 405us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2123/5000
6400/6400 [========

6400/6400 [==============================] - 2s 386us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2190/5000
6400/6400 [==============================] - 2s 388us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2191/5000
6400/6400 [==============================] - 2s 385us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2192/5000
6400/6400 [==============================] - 2s 384us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2193/5000
6400/6400 [==============================] - 2s 387us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2194/5000
6400/6400 [==============================] - 2s 387us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2195/5000
6400/6400 [==============================] - 2s 388us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2196/5000
6400/6400 [==============================] - 3s 392us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2197/5000
6400/6400 [==============================] - 3s 429us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2198/5000
6400/6400 [========

6400/6400 [==============================] - 2s 388us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2265/5000
6400/6400 [==============================] - 3s 412us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2266/5000
6400/6400 [==============================] - 3s 477us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2267/5000
6400/6400 [==============================] - 2s 384us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2268/5000
6400/6400 [==============================] - 3s 523us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2269/5000
6400/6400 [==============================] - 3s 397us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2270/5000
6400/6400 [==============================] - 3s 396us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2271/5000
6400/6400 [==============================] - 3s 391us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2272/5000
6400/6400 [==============================] - 3s 411us/step - loss: 0.0094 - val_loss: 0.0094
Epoch 2273/5000
6400/6400 [========

KeyboardInterrupt: 

In [ ]:
Names = ["on_amp", "off_amp", "on_phase", "off_phase", "on_resonance",
                      "off_resonance", "on_loss", "off_loss"]
input_on_amp = Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_off_amp =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_on_phase =Input(shape=(Nd_mat["Phase [rad]"].shape[2]//2,1))
input_off_phase =Input(shape=(Nd_mat["Phase [rad]"].shape[2]//2,1))
input_on_resonance =Input(shape=(Nd_mat["Phase [rad]"].shape[2]//2,1))
input_off_resonance = Input(shape=(Nd_mat["Phase [rad]"].shape[2]//2,1))
input_on_loss =Input(shape=(Nd_mat["Phase [rad]"].shape[2]//2,1))
input_off_loss =Input(shape=(Nd_mat["Phase [rad]"].shape[2]//2,1))


In [ ]:
values = [input_on_amp,input_off_amp,input_on_phase,input_off_phase, input_on_resonance, input_off_resonance,
         input_on_loss,input_off_loss]
Names = [None] * 8
for i, values in enumerate(values):
    Names[i] = network1(values)

In [ ]:
model = Model(inputs=[input_off_amp], 
              outputs=)

model.compile(optimizer=Adam(3e-5),loss='mse')#

In [ ]:
input_off_amp =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
out = Bidirectional(LSTM(24, return_sequences=False))(input_off_amp)
en_out_a_off = Dense(8, activation='relu')(out)
x = RepeatVector(64)(en_out_a_off)
#x = TimeDistributed(Dense(1, activation='linear'))(en_out_a_off)
x = LSTM(24, activation= "sigmoid", return_sequences=True)(x)
x = TimeDistributed(Dense(1, activation='linear'))(x)
model = Model(inputs=[input_off_amp], 
              outputs=[x])

model.compile(optimizer=Adam(3e-5),loss='mse')#

In [ ]:
input_on_amp = Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_off_amp =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_on_phase =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_off_phase =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_on_resonance =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_off_resonance = Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_on_loss =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))
input_off_loss =Input(shape=(Nd_mat["Phase [rad]"].shape[2],1))


en_out_a_on = encode_layer(3,32,input_on_amp)
en_out_a_on = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_a_on)
en_out_a_off = encode_layer(3,32,input_off_amp)
en_out_a_off = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_a_off)
en_out_p_on = encode_layer(3,32,input_on_phase)
en_out_p_on = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_p_on)
en_out_p_off = encode_layer(3,32,input_off_phase)
en_out_p_off = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_p_off)
en_out_r_on = encode_layer(3,32,input_on_resonance)
en_out_r_on = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_r_on)
en_out_r_off = encode_layer(3,32,input_off_resonance)
en_out_r_off = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_r_off)
en_out_l_on = encode_layer(3,32,input_on_loss)
en_out_l_on = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_l_on)
en_out_l_off = encode_layer(3,32,input_off_loss)
en_out_l_off = Dense(4, activation='relu',activity_regularizer=l1(10e-4))(en_out_l_off)



x = keras.layers.concatenate([en_out_a_on, en_out_a_off, en_out_p_on, en_out_p_off,
                              en_out_r_on, en_out_r_off, en_out_l_on, en_out_l_off])

x = Dense(12, activation='relu',activity_regularizer=l1(10e-5))(x)

#de_out_a_on = decode_layer(3,16,x)
#de_out_a_off =decode_layer(3,16,x)
#de_out_p_on = decode_layer(3,16,x)
#de_out_p_off =decode_layer(3,16,x)
#de_out_r_on = decode_layer(3,16,x)
#de_out_r_off =decode_layer(3,16,x)
#de_out_l_on = decode_layer(3,16,x)
#de_out_l_off =decode_layer(3,16,x)

model = Model(inputs=[input_on_amp, input_off_amp, input_on_phase, input_off_phase, input_on_resonance,
                      input_off_resonance, input_on_loss, input_off_loss] 
              outputs=[de_out_a_on, de_out_a_off, de_out_p_on, de_out_p_off, de_out_r_on, de_out_r_off,
                       de_out_l_on, de_out_l_off])

model.compile(optimizer=Adam(3e-5),loss='mse')#

In [ ]:
np.sum(~np.isfinite(amp_off))

In [ ]:
Path = 'amp_lay5'
make_folder(Path)
tbCallBack = keras.callbacks.TensorBoard(
    log_dir='./' + Path, histogram_freq=0, write_graph=True, write_images=True)
filepath = './' + Path + '/weights.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(filepath, verbose=0, save_best_only=False,
                                             save_weights_only=False, mode='min', period=1)
#from keras.utils import multi_gpu_model

# Replicates `model` on 2 GPUs.
# This assumes that your machine has 2 available GPUs.
#parallel_model = multi_gpu_model(model, gpus=2)
#parallel_model.compile(optimizer=Adam(3e-5),loss='mse')
model.fit([amp_off], 
          [amp_off],
          validation_data=([amp_off], 
          [amp_off]),
          epochs=5000,batch_size=1024, callbacks=[tbCallBack, checkpoint])


In [ ]:
model.save('./' + Path + '/end')

In [ ]:
dc_vec[0:128].shape

In [ ]:
model.summary()

In [ ]:
e = Nd_mat["Amplitude [V]"][:,:,:,1,2].reshape(-1,128)

h5_sho_spec_inds = px.hdf_utils.getAuxData(h5_sho_fit, auxDataName='Spectroscopic_Indices')[0]
h5_sho_spec_vals = px.hdf_utils.getAuxData(h5_sho_fit, auxDataName='Spectroscopic_Values')[0]

voltage_reshape = h5_sho_spec_vals[1] + h5_sho_spec_vals[2]

voltage_reshape.shape

dc_vec=dc_vec[::124]
plt.plot(dc_vec[0:128],amp_off[100])

In [ ]:
h5_file = h5_main.file
h5_spec_vals = h5_file[px.io.hdf_utils.get_attr(h5_main, 'Spectroscopic_Values')]
 
dc_vec = np.squeeze(h5_spec_vals[h5_spec_vals.attrs['DC_Offset']])


In [ ]:


model = Model(inputs=input_off_amp, outputs=out)


In [ ]:
type(e)

In [ ]:
model.summary()

In [ ]:
from keras.utils import multi_gpu_model

# Replicates `model` on 8 GPUs.
# This assumes that your machine has 8 available GPUs.
parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.compile(loss='categorical_crossentropy',
                       optimizer='rmsprop')

# This `fit` call will be distributed on 8 GPUs.
# Since the batch size is 256, each GPU will process 32 samples.
parallel_model.fit(x, y, epochs=20, batch_size=256)